In [2]:
import requests
import pandas as pd
from datetime import date
import teradatasql
from megatron import saving as m
import json

# connect to Teradata with a query to load in deal_uuids to check
con = teradatasql.connect(host='tdwd.group.on',user='mdijkhof',password='seinfeld66P&')
query = "SELECT DISTINCT opp2.deal_uuid FROM dwh_load_sf_view.sf_opportunity_1 AS opp JOIN dwh_load_sf_view.sf_opportunity_2 AS opp2 ON 1=1 AND opp.id = opp2.id WHERE 1=1 AND opp.feature_country in (/*'GB','IE','NL','BE','FR','ES','IT',*/'PL','DE','AE','AU') AND opp.category_v3 NOT IN ('Travel','Goods') AND opp.closedate >= '2022-01-01' AND deal_state IS NOT NULL"


# convert deal_uuids from query to list
df = pd.read_sql(query,con)
deal_uuid = df['deal_uuid'].tolist()

print(str(len(deal_uuid))+' number of deal_uuids')

deal_state = []

n = 0

# loop over deal_uuids in list and find the status in deal catalog
for deal in deal_uuid:
    n = n+1
    try:
        url = f'http://deal-catalog.snc1/deal_catalog/v2/deals/{deal}?clientId=f183e5fbee1bb4cd-display-ads'
        response = requests.get(url)
        data = response.json()
        
        # print(json.dumps(data,indent=4,sort_keys=False))
        
        status = data['deal']['status']
        deal_state.append(status)
        print(str(n)+' of '+str(len(deal_uuid))+': ' +deal+' processed')
        
    except:
        print(str(n)+' of '+str(len(deal_uuid))+': ' +deal+' does not exist')
        pass

# create a dataset with 3 columns
data = {
    'deal_uuid': deal_uuid, 
    'deal_state': deal_state, 
    'load_date': str(date.today())
    }

# push data to Teradata table
data_table = pd.DataFrame(data)
m.df_to_sql(data_table,table_name='md_deal_state_check',  adapter='teradata')

print("Success")


c:\Users\markd\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


13148 number of deal_uuids
1 of 13148: ce6496c9-c272-481b-bf0f-9affa4fbe459 processed
2 of 13148: 4b073f02-aee4-4c59-826a-93a99eac442b processed
3 of 13148: e3bb81bf-83d1-493d-a8d7-d7c7982c2ab8 processed
4 of 13148: 6a2aad0f-05a1-4ce3-bc67-4b922fb274e0 processed
5 of 13148: 3b03fb15-8544-4b04-ab3a-3999591e55de processed
6 of 13148: 03b2c95c-d8aa-4352-a562-d4e35300afb2 processed
7 of 13148: 7a0032c9-b088-410e-9333-52f5aed407b1 processed
8 of 13148: 782ac831-e1a8-4e7f-b208-4373cc8b4e13 processed
9 of 13148: 92247b65-5e4e-4e18-bb65-2b461a1b0d8a processed
10 of 13148: 903b45b7-1db4-46e1-9cf0-43133df79aaf processed
11 of 13148: 094950a2-9064-4d73-9d53-31a04a4d64a1 processed
12 of 13148: 15afe00b-d465-4209-a61e-9b9470a4cf4c processed
13 of 13148: 9e043b20-bd76-4488-932a-431b1994a561 processed
14 of 13148: f3cd298f-5ad6-47d5-b2cb-99d4e02f1d9c processed
15 of 13148: 6e3974d4-1dd0-4ea5-bcbf-114c0c23c7bd processed
16 of 13148: 69db2636-2380-42a7-82e3-5b7d0b3bfd53 processed
17 of 13148: 072775ac-